In [1]:
!pip install keras

In [5]:
!pip install tensorflow

INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 237.4 kB/s eta 0:00:0000:0100:15
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 365.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 638.6 kB/s eta 0:00:000:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 648.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 789.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 664.6 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 318.8 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout,MaxPooling1D, Flatten,BatchNormalization, GaussianNoise,Conv1D
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.utils import compute_class_weight
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, save_model, load_model

In [47]:
#this was created in general/diagnosis_making notebook
diag = pd.read_csv("../general/ground_truth.csv").drop("Unnamed: 0", axis=1)

Below we are combining several clinical datasets.

In [48]:
demo = pd.read_csv("PTDEMOG.csv").rename(columns={"PHASE":"Phase"})

In [102]:
demo

,Phase,PTID,RID,VISCODE,VISCODE2,VISDATE,PTSOURCE,PTGENDER,PTDOB,PTDOBYY,...,PTBIRPR,PTBIRGR,ID,SITEID,USERDATE,USERDATE2,DD_CRF_VERSION_LABEL,LANGUAGE_CODE,HAS_QC_ERROR,update_stamp
0,ADNI1,011_S_0002,2,sc,sc,2005-08-17,1,1.0,04/1931,1931.0,...,NaN,NaN,18,107,2005-08-17,NaN,NaN,NaN,NaN,2005-08-17 00:00:00.0
1,ADNI1,022_S_0001,1,f,f,2005-08-18,1,2.0,12/1944,1944.0,...,NaN,NaN,20,10,2005-08-18,NaN,NaN,NaN,NaN,2005-08-18 00:00:00.0
2,ADNI1,011_S_0003,3,sc,sc,2005-08-18,1,1.0,05/1924,1924.0,...,NaN,NaN,22,107,2005-08-18,NaN,NaN,NaN,NaN,2005-08-18 00:00:00.0
3,ADNI1,022_S_0004,4,sc,sc,2005-08-18,1,1.0,01/1938,1938.0,...,NaN,NaN,24,10,2005-08-18,NaN,NaN,NaN,NaN,2005-08-18 00:00:00.0
4,ADNI1,011_S_0005,5,sc,sc,2005-08-23,1,1.0,12/1931,1931.0,...,NaN,NaN,26,107,2005-08-23,NaN,NaN,NaN,NaN,2005-08-23 00:00:00.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5462,ADNI4,033_S_10326,10326,4_sc,sc,2024-10-29,1,1.0,09/1953,1953.0,...,1.0,1.0,46170,33,2024-11-04,2024-11-04,v1,e,0.0,2024-11-05 00:42:56.0
5463,ADNI4,082_S_6283,6283,4_init,m72,2024-07-29,1,2.0,08/1951,1951.0,...,1.0,1.0,46191,82,2024-11-04,2024-11-04,v1,e,0.0,2024-11-05 00:42:56.0
5464,ADNI4,082_S_10153,10153,4_sc,sc,2024-07-16,1,1.0,04/1944,1944.0,...,0.0,0.0,46197,82,2024-11-04,2024-11-04,v1,e,1.0,2024-11-05 00:42:56.0
5465,ADNI4,003_S_4644,4644,4_init,m144,2024-08-07,1,2.0,08/1944,1944.0,...,1.0,0.0,46248,3,2024-11-04,2024-11-04,v1,e,0.0,2024-11-05 00:42:56.0


In [103]:
len(demo['PTID'].unique())

4392

In [49]:
neuro = pd.read_csv("NEUROEXM.csv").rename(columns={"PHASE":"Phase"})

In [104]:
len(neuro['PTID'].unique())

3762

In [50]:
neuro.columns

Index(['Phase', 'PTID', 'RID', 'VISCODE', 'VISCODE2', 'VISDATE', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 'NXOTHER',
       'NXABNORM', 'ID', 'SITEID', 'USERDATE', 'USERDATE2',
       'DD_CRF_VERSION_LABEL', 'LANGUAGE_CODE', 'HAS_QC_ERROR',
       'update_stamp'],
      dtype='object')

In [ ]:
clinical = pd.read_csv("ADSP_PHC_COGN.csv").rename(columns={"PHASE":"Phase"})

In [63]:
clinical

,RID,SUBJID,Phase,VISCODE,VISCODE2,EXAMDATE,PHC_Visit,PHC_Sex,PHC_Education,PHC_Ethnicity,...,PHC_MEM_PreciseFilter,PHC_EXF,PHC_EXF_SE,PHC_EXF_PreciseFilter,PHC_LAN,PHC_LAN_SE,PHC_LAN_PreciseFilter,PHC_VSP,PHC_VSP_SE,PHC_VSP_PreciseFilter
0,2,ADNI_011_S_0002,ADNI1,bl,bl,2005-09-08,1,1.0,16.0,2.0,...,1,0.254,0.340,1.0,0.293,0.295,1.0,0.963,0.658,0.0
1,2,ADNI_011_S_0002,ADNI1,m06,m06,2006-03-06,2,1.0,16.0,2.0,...,1,0.278,0.358,1.0,0.535,0.307,1.0,-0.333,0.464,1.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,3,1.0,16.0,2.0,...,1,0.419,0.349,1.0,0.535,0.307,1.0,0.963,0.658,0.0
3,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,4,1.0,16.0,2.0,...,1,0.066,0.313,1.0,0.620,0.342,1.0,0.264,0.547,1.0
4,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,5,1.0,16.0,2.0,...,1,0.002,0.336,1.0,0.460,0.374,1.0,0.264,0.547,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12528,7118,NaN,ADNI3,sc,sc,2022-11-11,1,2.0,16.0,2.0,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12529,7119,NaN,ADNI3,sc,sc,2023-01-12,1,1.0,13.0,NaN,...,1,0.503,0.746,0.0,0.071,0.645,0.0,0.547,0.801,0.0
12530,7120,NaN,ADNI3,sc,sc,2023-01-19,1,1.0,16.0,2.0,...,1,-0.574,0.654,0.0,0.602,0.686,0.0,0.547,0.801,0.0
12531,7121,NaN,ADNI3,sc,sc,2023-02-09,1,1.0,17.0,2.0,...,1,0.503,0.746,0.0,0.602,0.686,0.0,0.547,0.801,0.0


In [66]:
len(clinical.SUBJID.unique())

1565

In [67]:
clinical.SUBJID.unique()

array(['ADNI_011_S_0002', 'G-ADNI-AN000439', 'G-ADNI-AN000446', ...,
       'G-ADNI-AN000355', 'G-ADNI-AN000321', 'G-ADNI-AN000309'],
      dtype=object)

In [52]:
clinical.head()

,RID,SUBJID,Phase,VISCODE,VISCODE2,EXAMDATE,PHC_Visit,PHC_Sex,PHC_Education,PHC_Ethnicity,...,PHC_MEM_PreciseFilter,PHC_EXF,PHC_EXF_SE,PHC_EXF_PreciseFilter,PHC_LAN,PHC_LAN_SE,PHC_LAN_PreciseFilter,PHC_VSP,PHC_VSP_SE,PHC_VSP_PreciseFilter
0,2,ADNI_011_S_0002,ADNI1,bl,bl,2005-09-08,1,1.0,16.0,2.0,...,1,0.254,0.340,1.0,0.293,0.295,1.0,0.963,0.658,0.0
1,2,ADNI_011_S_0002,ADNI1,m06,m06,2006-03-06,2,1.0,16.0,2.0,...,1,0.278,0.358,1.0,0.535,0.307,1.0,-0.333,0.464,1.0
2,2,ADNI_011_S_0002,ADNI1,m36,m36,2008-08-27,3,1.0,16.0,2.0,...,1,0.419,0.349,1.0,0.535,0.307,1.0,0.963,0.658,0.0
3,2,ADNI_011_S_0002,ADNIGO,m60,m60,2010-09-22,4,1.0,16.0,2.0,...,1,0.066,0.313,1.0,0.620,0.342,1.0,0.264,0.547,1.0
4,2,ADNI_011_S_0002,ADNI2,v06,m72,2011-09-19,5,1.0,16.0,2.0,...,1,0.002,0.336,1.0,0.460,0.374,1.0,0.264,0.547,1.0


In [53]:
diag["Subject"].value_counts()

Subject
031_S_0618    4
003_S_0981    4
057_S_0934    4
141_S_1378    4
006_S_0731    4
             ..
109_S_2278    1
068_S_2194    1
068_S_2184    1
002_S_2073    1
013_S_7110    1
Name: count, Length: 2738, dtype: int64

In [113]:
comb = pd.read_csv("DXSUM.csv")[["RID", "PTID" , "PHASE"]].rename(columns={'PHASE': 'Phase'})

In [115]:
comb

,RID,PTID,Phase
0,2,011_S_0002,ADNI1
1,3,011_S_0003,ADNI1
2,5,011_S_0005,ADNI1
3,8,011_S_0008,ADNI1
4,7,022_S_0007,ADNI1
...,...,...,...
14313,10105,073_S_10105,ADNI4
14314,10172,073_S_10172,ADNI4
14315,10088,073_S_10088,ADNI4
14316,10380,114_S_10380,ADNI4


In [116]:
len(comb['PTID'].unique())

3295

In [117]:
m = comb.merge(demo, on = ["RID", "Phase"]).merge(neuro,on = ["RID", "Phase"]).merge(clinical,on = ["RID", "Phase"]).drop_duplicates()

In [118]:
m.columns = [c[:-2] if str(c).endswith(('_x','_y')) else c for c in m.columns]

m = m.loc[:,~m.columns.duplicated()]

In [119]:
diag = diag.rename(columns = {"Subject": "PTID"})

In [120]:
m = m.merge(diag, on = ["PTID", "Phase"])

In [121]:
m["PTID"].value_counts()

PTID
067_S_4767    18
141_S_0697    15
137_S_0973    14
094_S_1417    14
116_S_0752    14
              ..
002_S_6021     1
130_S_6028     1
037_S_6032     1
141_S_6041     1
177_S_6338     1
Name: count, Length: 2334, dtype: int64

In [122]:
t = m

In [123]:
t = t.drop(["ID",  "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2",
            "update_stamp", "PTDOB", "PTSOURCE","PHC_Diagnosis","VISDATE"], axis=1) 

In [124]:
t.columns

Index(['RID', 'PTID', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND', 'PTMARRY',
       'PTEDUCAT', 'PTWORKHS', 'PTWORK',
       ...
       'PHC_EXF', 'PHC_EXF_SE', 'PHC_EXF_PreciseFilter', 'PHC_LAN',
       'PHC_LAN_SE', 'PHC_LAN_PreciseFilter', 'PHC_VSP', 'PHC_VSP_SE',
       'PHC_VSP_PreciseFilter', 'Group'],
      dtype='object', length=105)

In [125]:
t = t.fillna(-4)
t = t.replace("-4", -4)
cols_to_delete = t.columns[(t == -4).sum()/len(t) > .70]
t.drop(cols_to_delete, axis = 1, inplace = True)

In [126]:
len(t.columns)

52

In [127]:
t

,RID,PTID,Phase,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTNOTRT,PTRTYR,...,PHC_EXF,PHC_EXF_SE,PHC_EXF_PreciseFilter,PHC_LAN,PHC_LAN_SE,PHC_LAN_PreciseFilter,PHC_VSP,PHC_VSP_SE,PHC_VSP_PreciseFilter,Group
0,3,011_S_0003,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,1989.0,...,-0.669,0.298,1.0,-0.516,0.238,1.0,-0.237,0.476,1.0,2.0
1,3,011_S_0003,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,1989.0,...,-0.137,0.336,1.0,-0.516,0.249,1.0,-0.305,0.484,1.0,2.0
2,3,011_S_0003,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,1989.0,...,-1.047,0.282,1.0,-0.828,0.231,1.0,0.128,0.539,1.0,2.0
3,3,011_S_0003,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,1989.0,...,-0.637,0.290,1.0,-0.756,0.260,1.0,0.963,0.658,0.0,2.0
4,5,011_S_0005,ADNI1,1.0,1931.0,1.0,1.0,16.0,1.0,1987.0,...,0.615,0.347,1.0,0.345,0.278,1.0,0.963,0.658,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7273,7120,035_S_7120,ADNI3,1.0,1958.0,1.0,4.0,16.0,0.0,-4.0,...,-0.574,0.654,0.0,0.602,0.686,0.0,0.547,0.801,0.0,2.0
7274,7117,082_S_7117,ADNI3,2.0,1953.0,1.0,4.0,18.0,1.0,2017.0,...,1.230,0.451,1.0,0.886,0.418,1.0,0.963,0.658,0.0,1.0
7275,7110,013_S_7110,ADNI3,1.0,1945.0,2.0,1.0,17.0,1.0,2017.0,...,0.503,0.746,0.0,0.602,0.686,0.0,0.547,0.801,0.0,1.0
7276,6949,016_S_6949,ADNI3,2.0,1951.0,1.0,1.0,14.0,1.0,2018.0,...,0.503,0.746,0.0,0.064,0.636,0.0,0.547,0.801,0.0,1.0


In [ ]:
# t["PTWORK"] = t["PTWORK"].str.lower().str.replace("housewife", "homemaker").str.replace("rn", "nurse").str.replace("bookeeper",
#                                                                                                                 #    "bookkeeper").str.replace("cpa", "accounting")

In [ ]:
# t["PTWORK"] = t["PTWORK"].fillna("-4").astype(str)

In [ ]:
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*teach.*$)', 'education')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*bookkeep.*$)', 'bookkeeper')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*wife.*$)', 'homemaker')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*educat.*$)', 'education')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*engineer.*$)', 'engineer')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*eingineering.*$)', 'engineer') 
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*computer programmer.*$)', 'engineer') 
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*nurs.*$)', 'nurse')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*manage.*$)', 'managment')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*therapist.*$)', 'therapist')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*sales.*$)', 'sales')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*admin.*$)', 'admin')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*account.*$)', 'accounting')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*real.*$)', 'real estate')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*secretary.*$)', 'secretary')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*professor.*$)', 'professor')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*chem.*$)', 'chemist')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*business.*$)', 'business')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*writ.*$)', 'writing')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*psych.*$)', 'psychology')
# t['PTWORK'] = t['PTWORK'].str.replace(r'(^.*analys.*$)', 'analyst')

In [ ]:
# cond = t['PTWORK'].value_counts()
# threshold = 10
# t['PTWORK'] = np.where(t['PTWORK'].isin(cond.index[cond >= threshold ]), t['PTWORK'], 'other')

In [128]:
categorical = ['PTGENDER', 
 'PTHOME',
 'PTMARRY',
 'PTEDUCAT',
 'PTPLANG',
 'NXVISUAL',
 'PTNOTRT',
 'NXTREMOR',
 'NXAUDITO',
 'PTHAND']

In [129]:
quant = ['PTDOBYY',
 'PHC_MEM',
 'PHC_EXF',
 'PTRACCAT',
 'AGE',
 'PTADDX',
 'PTETHCAT',
 'PTCOGBEG',
 'PHC_VSP',
 'PHC_LAN']

In [130]:
text = ["PTWORK", "CMMED"]

In [131]:
cols_left = list(set(t.columns) - set(categorical) - set(text)  - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[cols_left]

,NXMOTOR,PHC_VSP,NXHEEL,NXNERVE,PHC_Education,NXSENSOR,PTDOBYY,PHC_Race,EXAMDATE,PTTLANG,...,PHC_Age_Cognition,PHC_LAN_SE,PHC_VSP_PreciseFilter,PHC_LAN_PreciseFilter,PHC_EXF_SE,NXTENDON,PHC_EXF_PreciseFilter,PTETHCAT,PHC_MEM,PHC_VSP_SE
0,1.0,-0.237,1.0,1.0,18.0,1.0,1924.0,5.0,2005-09-12,1.0,...,81.366188,0.238,1.0,1.0,0.298,1.0,1.0,2.0,-0.955,0.476
1,1.0,-0.305,1.0,1.0,18.0,1.0,1924.0,5.0,2006-03-13,1.0,...,81.864476,0.249,1.0,1.0,0.336,1.0,1.0,2.0,-0.657,0.484
2,1.0,0.128,1.0,1.0,18.0,1.0,1924.0,5.0,2006-09-12,1.0,...,82.365503,0.231,1.0,1.0,0.282,1.0,1.0,2.0,-0.917,0.539
3,1.0,0.963,1.0,1.0,18.0,1.0,1924.0,5.0,2007-09-12,1.0,...,83.364819,0.260,0.0,1.0,0.290,1.0,1.0,2.0,-1.396,0.658
4,1.0,0.963,1.0,1.0,16.0,2.0,1931.0,5.0,2005-09-07,1.0,...,73.768652,0.278,0.0,1.0,0.347,1.0,1.0,2.0,0.658,0.658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7273,1.0,0.547,1.0,1.0,16.0,1.0,1958.0,5.0,2023-01-19,1.0,...,64.887064,0.686,0.0,0.0,0.654,2.0,0.0,2.0,-0.688,0.801
7274,1.0,0.963,1.0,1.0,18.0,1.0,1953.0,3.0,2023-01-12,1.0,...,69.196441,0.418,0.0,1.0,0.451,2.0,1.0,2.0,1.248,0.658
7275,1.0,0.547,1.0,1.0,17.0,1.0,1945.0,5.0,2023-01-24,1.0,...,77.396304,0.686,0.0,0.0,0.746,1.0,0.0,2.0,0.390,0.801
7276,1.0,0.547,1.0,1.0,14.0,1.0,1951.0,5.0,2021-05-17,2.0,...,70.212183,0.636,0.0,0.0,0.746,1.0,0.0,1.0,0.633,0.801


In [132]:
for col in cols_left:
    if len(t[col].value_counts()) < 10:
        print(col)
        categorical.append(col)

NXMOTOR
NXHEEL
NXNERVE
NXSENSOR
PHC_Race
PTTLANG
NXCONSCI
NXGAIT
PHC_Sex
PHC_Ethnicity
PHC_Visit
NXABNORM
NXFINGER
NXPLANTA
PTRACCAT
NXOTHER
PHC_MEM_PreciseFilter
PHC_VSP_PreciseFilter
PHC_LAN_PreciseFilter
NXTENDON
PHC_EXF_PreciseFilter
PTETHCAT


In [133]:
t

,RID,PTID,Phase,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTNOTRT,PTRTYR,...,PHC_EXF,PHC_EXF_SE,PHC_EXF_PreciseFilter,PHC_LAN,PHC_LAN_SE,PHC_LAN_PreciseFilter,PHC_VSP,PHC_VSP_SE,PHC_VSP_PreciseFilter,Group
0,3,011_S_0003,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,1989.0,...,-0.669,0.298,1.0,-0.516,0.238,1.0,-0.237,0.476,1.0,2.0
1,3,011_S_0003,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,1989.0,...,-0.137,0.336,1.0,-0.516,0.249,1.0,-0.305,0.484,1.0,2.0
2,3,011_S_0003,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,1989.0,...,-1.047,0.282,1.0,-0.828,0.231,1.0,0.128,0.539,1.0,2.0
3,3,011_S_0003,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,1989.0,...,-0.637,0.290,1.0,-0.756,0.260,1.0,0.963,0.658,0.0,2.0
4,5,011_S_0005,ADNI1,1.0,1931.0,1.0,1.0,16.0,1.0,1987.0,...,0.615,0.347,1.0,0.345,0.278,1.0,0.963,0.658,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7273,7120,035_S_7120,ADNI3,1.0,1958.0,1.0,4.0,16.0,0.0,-4.0,...,-0.574,0.654,0.0,0.602,0.686,0.0,0.547,0.801,0.0,2.0
7274,7117,082_S_7117,ADNI3,2.0,1953.0,1.0,4.0,18.0,1.0,2017.0,...,1.230,0.451,1.0,0.886,0.418,1.0,0.963,0.658,0.0,1.0
7275,7110,013_S_7110,ADNI3,1.0,1945.0,2.0,1.0,17.0,1.0,2017.0,...,0.503,0.746,0.0,0.602,0.686,0.0,0.547,0.801,0.0,1.0
7276,6949,016_S_6949,ADNI3,2.0,1951.0,1.0,1.0,14.0,1.0,2018.0,...,0.503,0.746,0.0,0.064,0.636,0.0,0.547,0.801,0.0,1.0


In [134]:
to_del = ["PTRTYR", "EXAMDATE", "SUBJID"]
t = t.drop(to_del, axis=1)

In [135]:
quant = list(set(cols_left) - set(categorical) - set(text)  -set(to_del) - set(["label", "Group","GROUP", "Phase", "RID", "PTID"]))
t[quant]

,PHC_VSP,PTDOBYY,PTCOGBEG,PHC_Age_Cognition,PHC_LAN_SE,PHC_EXF_SE,PTADDX,PHC_Education,PHC_LAN,PHC_MEM,PHC_VSP_SE,PHC_MEM_SE,PHC_EXF
0,-0.237,1924.0,-4.0,81.366188,0.238,0.298,-4.0,18.0,-0.516,-0.955,0.476,0.200,-0.669
1,-0.305,1924.0,-4.0,81.864476,0.249,0.336,-4.0,18.0,-0.516,-0.657,0.484,0.182,-0.137
2,0.128,1924.0,-4.0,82.365503,0.231,0.282,-4.0,18.0,-0.828,-0.917,0.539,0.166,-1.047
3,0.963,1924.0,-4.0,83.364819,0.260,0.290,-4.0,18.0,-0.756,-1.396,0.658,0.156,-0.637
4,0.963,1931.0,-4.0,73.768652,0.278,0.347,-4.0,16.0,0.345,0.658,0.658,0.181,0.615
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7273,0.547,1958.0,2020.0,64.887064,0.686,0.654,-4.0,16.0,0.602,-0.688,0.801,0.257,-0.574
7274,0.963,1953.0,2020.0,69.196441,0.418,0.451,9999.0,18.0,0.886,1.248,0.658,0.188,1.230
7275,0.547,1945.0,2017.0,77.396304,0.686,0.746,9999.0,17.0,0.602,0.390,0.801,0.380,0.503
7276,0.547,1951.0,2018.0,70.212183,0.636,0.746,9999.0,14.0,0.064,0.633,0.801,0.401,0.503


In [136]:
cols_left = list(set(cols_left) - set(categorical) - set(text) - set(quant) - set(to_del))

In [137]:
#after reviewing the meaning of each column, these are the final ones
l = ['RID', 'PTID', 'Group', 'Phase', 'PTGENDER', 'PTDOBYY', 'PTHAND',
       'PTMARRY', 'PTEDUCAT', 'PTNOTRT', 'PTHOME', 'PTTLANG',
       'PTPLANG', 'PTCOGBEG', 'PTETHCAT', 'PTRACCAT', 'NXVISUAL',
       'NXAUDITO', 'NXTREMOR', 'NXCONSCI', 'NXNERVE', 'NXMOTOR', 'NXFINGER',
       'NXHEEL', 'NXSENSOR', 'NXTENDON', 'NXPLANTA', 'NXGAIT', 
       'NXABNORM',  'PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']

In [138]:
t[l]

,RID,PTID,Group,Phase,PTGENDER,PTDOBYY,PTHAND,PTMARRY,PTEDUCAT,PTNOTRT,...,NXHEEL,NXSENSOR,NXTENDON,NXPLANTA,NXGAIT,NXABNORM,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP
0,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.955,-0.669,-0.516,-0.237
1,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.657,-0.137,-0.516,-0.305
2,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.917,-1.047,-0.828,0.128
3,3,011_S_0003,2.0,ADNI1,1.0,1924.0,1.0,1.0,18.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,-1.396,-0.637,-0.756,0.963
4,5,011_S_0005,0.0,ADNI1,1.0,1931.0,1.0,1.0,16.0,1.0,...,1.0,2.0,1.0,1.0,1.0,1.0,0.658,0.615,0.345,0.963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7273,7120,035_S_7120,2.0,ADNI3,1.0,1958.0,1.0,4.0,16.0,0.0,...,1.0,1.0,2.0,1.0,1.0,1.0,-0.688,-0.574,0.602,0.547
7274,7117,082_S_7117,1.0,ADNI3,2.0,1953.0,1.0,4.0,18.0,1.0,...,1.0,1.0,2.0,1.0,1.0,1.0,1.248,1.230,0.886,0.963
7275,7110,013_S_7110,1.0,ADNI3,1.0,1945.0,2.0,1.0,17.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.390,0.503,0.602,0.547
7276,6949,016_S_6949,1.0,ADNI3,2.0,1951.0,1.0,1.0,14.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.633,0.503,0.064,0.547


In [139]:
dfs = []

In [140]:
for col in categorical:
    dfs.append(pd.get_dummies(t[col], prefix = col))

In [141]:
cat = pd.concat(dfs, axis=1)

In [142]:
t[quant]

,PHC_VSP,PTDOBYY,PTCOGBEG,PHC_Age_Cognition,PHC_LAN_SE,PHC_EXF_SE,PTADDX,PHC_Education,PHC_LAN,PHC_MEM,PHC_VSP_SE,PHC_MEM_SE,PHC_EXF
0,-0.237,1924.0,-4.0,81.366188,0.238,0.298,-4.0,18.0,-0.516,-0.955,0.476,0.200,-0.669
1,-0.305,1924.0,-4.0,81.864476,0.249,0.336,-4.0,18.0,-0.516,-0.657,0.484,0.182,-0.137
2,0.128,1924.0,-4.0,82.365503,0.231,0.282,-4.0,18.0,-0.828,-0.917,0.539,0.166,-1.047
3,0.963,1924.0,-4.0,83.364819,0.260,0.290,-4.0,18.0,-0.756,-1.396,0.658,0.156,-0.637
4,0.963,1931.0,-4.0,73.768652,0.278,0.347,-4.0,16.0,0.345,0.658,0.658,0.181,0.615
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7273,0.547,1958.0,2020.0,64.887064,0.686,0.654,-4.0,16.0,0.602,-0.688,0.801,0.257,-0.574
7274,0.963,1953.0,2020.0,69.196441,0.418,0.451,9999.0,18.0,0.886,1.248,0.658,0.188,1.230
7275,0.547,1945.0,2017.0,77.396304,0.686,0.746,9999.0,17.0,0.602,0.390,0.801,0.380,0.503
7276,0.547,1951.0,2018.0,70.212183,0.636,0.746,9999.0,14.0,0.064,0.633,0.801,0.401,0.503


In [143]:
cat

,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,PTHOME_3.0,PTHOME_4.0,PTHOME_5.0,PTHOME_6.0,...,NXTENDON_-4.0,NXTENDON_1.0,NXTENDON_2.0,PHC_EXF_PreciseFilter_-4.0,PHC_EXF_PreciseFilter_0.0,PHC_EXF_PreciseFilter_1.0,PTETHCAT_-4.0,PTETHCAT_1.0,PTETHCAT_2.0,PTETHCAT_3.0
0,False,True,False,False,False,False,True,False,False,False,...,False,True,False,False,False,True,False,False,True,False
1,False,True,False,False,False,False,True,False,False,False,...,False,True,False,False,False,True,False,False,True,False
2,False,True,False,False,False,False,True,False,False,False,...,False,True,False,False,False,True,False,False,True,False
3,False,True,False,False,False,False,True,False,False,False,...,False,True,False,False,False,True,False,False,True,False
4,False,True,False,False,True,False,False,False,False,False,...,False,True,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7273,False,True,False,False,False,True,False,False,False,False,...,False,False,True,False,True,False,False,False,True,False
7274,False,False,True,False,True,False,False,False,False,False,...,False,False,True,False,False,True,False,False,True,False
7275,False,True,False,False,False,False,True,False,False,False,...,False,True,False,False,True,False,False,False,True,False
7276,False,False,True,False,False,True,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False


In [144]:
t[["PTID","RID", "Phase", "Group"]]

,PTID,RID,Phase,Group
0,011_S_0003,3,ADNI1,2.0
1,011_S_0003,3,ADNI1,2.0
2,011_S_0003,3,ADNI1,2.0
3,011_S_0003,3,ADNI1,2.0
4,011_S_0005,5,ADNI1,0.0
...,...,...,...,...
7273,035_S_7120,7120,ADNI3,2.0
7274,082_S_7117,7117,ADNI3,1.0
7275,013_S_7110,7110,ADNI3,1.0
7276,016_S_6949,6949,ADNI3,1.0


In [145]:
c = pd.concat([t[["PTID", "RID", "Phase", "Group"]].reset_index(), cat.reset_index(), t[quant].reset_index()], axis=1).drop("index", axis=1) #tex

In [146]:
c

,PTID,RID,Phase,Group,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,...,PHC_Age_Cognition,PHC_LAN_SE,PHC_EXF_SE,PTADDX,PHC_Education,PHC_LAN,PHC_MEM,PHC_VSP_SE,PHC_MEM_SE,PHC_EXF
0,011_S_0003,3,ADNI1,2.0,False,True,False,False,False,False,...,81.366188,0.238,0.298,-4.0,18.0,-0.516,-0.955,0.476,0.200,-0.669
1,011_S_0003,3,ADNI1,2.0,False,True,False,False,False,False,...,81.864476,0.249,0.336,-4.0,18.0,-0.516,-0.657,0.484,0.182,-0.137
2,011_S_0003,3,ADNI1,2.0,False,True,False,False,False,False,...,82.365503,0.231,0.282,-4.0,18.0,-0.828,-0.917,0.539,0.166,-1.047
3,011_S_0003,3,ADNI1,2.0,False,True,False,False,False,False,...,83.364819,0.260,0.290,-4.0,18.0,-0.756,-1.396,0.658,0.156,-0.637
4,011_S_0005,5,ADNI1,0.0,False,True,False,False,True,False,...,73.768652,0.278,0.347,-4.0,16.0,0.345,0.658,0.658,0.181,0.615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7273,035_S_7120,7120,ADNI3,2.0,False,True,False,False,False,True,...,64.887064,0.686,0.654,-4.0,16.0,0.602,-0.688,0.801,0.257,-0.574
7274,082_S_7117,7117,ADNI3,1.0,False,False,True,False,True,False,...,69.196441,0.418,0.451,9999.0,18.0,0.886,1.248,0.658,0.188,1.230
7275,013_S_7110,7110,ADNI3,1.0,False,True,False,False,False,False,...,77.396304,0.686,0.746,9999.0,17.0,0.602,0.390,0.801,0.380,0.503
7276,016_S_6949,6949,ADNI3,1.0,False,False,True,False,False,True,...,70.212183,0.636,0.746,9999.0,14.0,0.064,0.633,0.801,0.401,0.503


In [147]:
#removing repeating subjects, taking the most recent diagnosis
c = c.groupby('PTID', 
                  group_keys=False).apply(lambda x: x.loc[x["Group"].astype(int).idxmax()]).drop("PTID", axis = 1).reset_index(inplace=False)

In [148]:
len(c)

2334

In [149]:
len(c['PTID'].unique())

2334

In [150]:
c.to_csv("clinical.csv")

In [ ]:
##### SKIP FIRST
#reading in the overlap test set
ts = pd.read_csv("overlap_test_set.csv").rename(columns={"subject": "PTID"})

#removing ids from the overlap test set
c = c[~c["PTID"].isin(list(ts["PTID"].values))]

In [227]:
cols = list(set(c.columns) - set(["PTID","RID","subject", "ID","GROUP", "Group", "label", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX_x","DX_y", "Unnamed: 0"]))
X = c[cols].values 
y = c["Group"].astype(int).values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [228]:
cols

['NXVISUAL_-4.0',
 'NXCONSCI_1.0',
 'PTRACCAT_5',
 'PTEDUCAT_13.0',
 'PTTLANG_2.0',
 'NXPLANTA_2.0',
 'PHC_EXF_PreciseFilter_1.0',
 'PHC_Visit_8',
 'NXPLANTA_-4.0',
 'PHC_MEM_PreciseFilter_1',
 'NXSENSOR_1.0',
 'PTEDUCAT_4.0',
 'PHC_EXF_PreciseFilter_0.0',
 'PTRACCAT_7',
 'PTPLANG_2.0',
 'NXHEEL_1.0',
 'PTRACCAT_3',
 'PTRACCAT_-4',
 'NXFINGER_1.0',
 'PHC_EXF_SE',
 'PTHOME_5.0',
 'PTEDUCAT_5.0',
 'NXABNORM_1.0',
 'PHC_MEM_SE',
 'PTEDUCAT_17.0',
 'PTGENDER_-4.0',
 'PTEDUCAT_20.0',
 'PTEDUCAT_11.0',
 'PTEDUCAT_7.0',
 'PTHOME_8.0',
 'PHC_Race_2.0',
 'PHC_EXF_PreciseFilter_-4.0',
 'PTHOME_3.0',
 'PHC_EXF',
 'PTRACCAT_1',
 'PHC_Ethnicity_1.0',
 'PHC_MEM',
 'PHC_Visit_5',
 'PTGENDER_1.0',
 'PTRACCAT_2',
 'NXTREMOR_-4.0',
 'NXMOTOR_1.0',
 'NXAUDITO_1.0',
 'PHC_LAN_PreciseFilter_-4.0',
 'NXVISUAL_2.0',
 'PHC_Education',
 'PTDOBYY',
 'PTHOME_2.0',
 'PHC_Race_1.0',
 'PTMARRY_-4.0',
 'PTEDUCAT_19.0',
 'PHC_Sex_1.0',
 'PTPLANG_3.0',
 'PHC_LAN_PreciseFilter_1.0',
 'PTNOTRT_1.0',
 'PHC_VSP',
 'NXVISU

In [229]:
X_train_df = pd.DataFrame(X_train, columns=cols)
y_train_df = pd.DataFrame(y_train)
X_test_df = pd.DataFrame(X_test, columns=cols)
y_test_df = pd.DataFrame(y_test)

# 保存为pkl文件
X_train_df.to_pickle("X_train_c.pkl")
y_train_df.to_pickle("y_train_c.pkl")
X_test_df.to_pickle("X_test_c.pkl")
y_test_df.to_pickle("y_test_c.pkl")
